In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [44]:
# Read file "hw1_data.tsv"
file = open('/content/drive/MyDrive/hw1_data.tsv', 'r')
ori_data = file.read().strip().split("\n")
data = []
for item in ori_data:
  item = item.split("\t")
  if len(item[0]) >0:
    data.append((item[0],item[1]))
print(len(data))

300


In [45]:
### Training data pre-processing
texts, labels = [], []
label2idx = {"0":0, "1":1}
for i, item in enumerate(data):
  text = item[0]

  ## Preprocessing (if you want to add, please add more)
  ################################################################################
  text = text.replace("  "," ") ## Replace double space
  text = text.replace(",", "") ## Replace comma to ""
  text = text.lower()  ## Lower cases
  ################################################################################
  label = label2idx[item[1]]

  texts.append(text)
  labels.append(label)

print("*"*50)
print("Total number of datasets")
print(len(texts))
print(len(labels))
print("*"*50)




**************************************************
Total number of datasets
300
300
**************************************************


In [46]:
#### Split into train/dev/test sets
from sklearn.model_selection import train_test_split
### Write a code for collecting samples for each class
################################################################################
pos,neg = [], []
train_texts, dev_texts, test_texts, train_labels, dev_labels, test_labels = [], [], [], [], [], []
for a,b in zip(texts,labels):
  if b == 1:
    pos.append((a,b))
  else:
    neg.append((a,b))

for a,b in pos[:50]:
  test_texts.append(a)
  test_labels.append(b)
for a,b in neg[:50]:
  test_texts.append(a)
  test_labels.append(b)

for a,b in pos[50:100]:
  dev_texts.append(a)
  dev_labels.append(b)
for a,b in neg[50:100]:
  dev_texts.append(a)
  dev_labels.append(b)

for a,b in pos[100:]:
  train_texts.append(a)
  train_labels.append(b)
for a,b in neg[100:]:
  train_texts.append(a)
  train_labels.append(b)
################################################################################


tmp = list(zip(train_texts,train_labels))
import random
random.shuffle(tmp)
train_text, train_labels = zip(*tmp)

tmp = list(zip(dev_texts,dev_labels))
import random
random.shuffle(tmp)
dev_texts, dev_labels = zip(*tmp)

tmp = list(zip(test_texts,test_labels))
import random
random.shuffle(tmp)
test_texts, test_labels = zip(*tmp)

print("Train Dataset Examples")
print(train_texts[:3])
print(train_labels[:3])
print("*"*50)
print(train_labels)
print(dev_labels)

Train Dataset Examples
['speak for it while it forces you to ponder anew what a movie can be ', 'enables shafer to navigate spaces both large ... and small ... with considerable aplomb ', 'takes chances that are bold by studio standards ']
(0, 1, 1)
**************************************************
(0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0)
(1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0)


In [51]:
## Construct a vocabulary
from collections import Counter

all_words = []
for item in train_texts:
  all_words += item.split()
for item in dev_texts:
  all_words += item.split()

## Build a dictionary that maps words to integers
counts = Counter(all_words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {'<pad>':0, "<unk>":1}
vocab_to_int.update({word: ii for ii, word in enumerate(vocab,2)})
print(vocab_to_int)
print(len(vocab_to_int))

{'<pad>': 0, '<unk>': 1, 'the': 2, 'and': 3, 'a': 4, 'to': 5, '.': 6, 'of': 7, 'it': 8, 'in': 9, 'that': 10, 'is': 11, "'s": 12, 'as': 13, 'for': 14, 'you': 15, 'with': 16, 'film': 17, 'one': 18, 'movie': 19, 'by': 20, 'this': 21, 'its': 22, '--': 23, '...': 24, 'have': 25, 'so': 26, 'good': 27, 'story': 28, 'most': 29, 'if': 30, "n't": 31, 'your': 32, 'time': 33, 'no': 34, 'more': 35, 'not': 36, 'be': 37, 'takes': 38, 'are': 39, 'way': 40, 'an': 41, 'little': 42, 'on': 43, 'about': 44, 'very': 45, '(': 46, ')': 47, 'own': 48, 'but': 49, 'i': 50, 'too': 51, 'can': 52, 'from': 53, 'some': 54, 'at': 55, '``': 56, 'picture': 57, 'or': 58, 'just': 59, 'worst': 60, 'when': 61, 'could': 62, 'his': 63, 'comedy': 64, 'what': 65, 'surprisingly': 66, 'does': 67, 'better': 68, 'old': 69, 'life': 70, 'into': 71, "''": 72, 'moments': 73, 'watching': 74, 'like': 75, 'recent': 76, 'all': 77, 'many': 78, 'their': 79, 'every': 80, 'hate': 81, 'work': 82, 'completely': 83, 'men': 84, 'small': 85, 'consi

In [52]:
import numpy as np
def encode_sentence(sentence):
    max_length = 50
    input_ids = []
    for item in sentence.split():
      if item in vocab_to_int:
        input_ids.append(vocab_to_int[item])
      else:
        input_ids.append(vocab_to_int['<unk>'])

    padding_length = max_length - len(input_ids)
    input_ids += [vocab_to_int['<pad>']] * padding_length
    return np.array(input_ids)

def encode_label(label):
    return np.array(label)



print("Train Dataset Encode Examples")
for a,b in zip(train_texts[:3],train_labels[:3]):
  print(encode_sentence(a),b)

Train Dataset Encode Examples
[195  14   8 196   8 197  15   5 198 199  65   4  19  52  37   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0] 0
[200 201   5 202 203 204 205  24   3  85  24  16  86  87   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0] 1
[ 38 206  10  39 207  20 208 209   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0] 1


In [53]:
import numpy as np

class SimpleNN:
    def __init__(self, vocab_size, embedding_dim, hidden_size, output_size):
        # Initialize weights and biases
        self.embedding_weights = np.random.rand(vocab_size, embedding_dim)  # Fixed embedding weights
        limit = np.sqrt(6 / (embedding_dim + hidden_size))
        self.W1 = np.random.uniform(-limit, limit, size=(embedding_dim, hidden_size))
        #self.W1 = np.random.rand(embedding_dim, hidden_size)   # Input to hidden weights
        limit = np.sqrt(6 / (hidden_size + output_size))
        self.W2 = np.random.uniform(-limit, limit, size=(hidden_size, output_size))
        #self.W2 = np.random.rand(hidden_size, output_size) # Hidden to output weights
        self.b1 = np.zeros((1, hidden_size))  # Hidden layer biases
        self.b2 = np.zeros((1, output_size))  # Output layer biases
        self.X = None

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def sigmoid_derivative(self, z):
        return z * (1 - z)

    def forward(self, X):
      # Forward pass
        ##############################################################################
        embedding = np.mean(self.embedding_weights[X], axis=0, keepdims=True)  # 임베딩 벡터 평균
        Z1 = np.dot(embedding, self.W1) + self.b1
        A1 = self.sigmoid(Z1)
        Z2 = np.dot(A1, self.W2) + self.b2
        output = self.sigmoid(Z2)

        self.A1 = A1  # Backward에서 사용하기 위해 저장
        return output
        ##############################################################################

    def compute_loss(self, y, output):
        # Compute binary cross-entropy loss
        return -np.sum(y * np.log(output) + (1 - y) * np.log(1 - output))

    def backward(self, X, y, output, learning_rate=0.01):
      ##############################################################################
        dZ2 = output - y

        dW2 = np.dot(self.A1.T, dZ2)
        db2 = np.sum(dZ2, axis=0, keepdims=True)

        dA1 = np.dot(dZ2, self.W2.T)
        dZ1 = dA1 * self.sigmoid_derivative(self.A1)

        embedding = self.embedding_weights[X].mean(axis=0).reshape(-1, 1)
        dW1 = np.dot(embedding, dZ1)
        db1 = np.sum(dZ1, axis=0, keepdims=True)

        self.W2 -= learning_rate * dW2
        self.b2 -= learning_rate * db2
        self.W1 -= learning_rate * dW1
        self.b1 -= learning_rate * db1
      ##############################################################################

In [54]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

def train(X, y, learning_rate=0.01):
    output = nn.forward(X)  # Forward pass
    loss = nn.compute_loss(y, output)  # Compute loss
    nn.backward(X, y, output, learning_rate)  # Backward pass
    return loss

def predict(x):
    output = nn.forward(x)
    return output, (output > 0.5).astype(int)  # Binary classification

# Initialize the neural network
vocab_size = len(vocab_to_int)  # Number of unique words in vocab
embedding_dim = 100  # Embedding dimension
hidden_size = 80  # Number of neurons in the hidden layer
output_size = 1  # One output (binary classification)
learning_rate = 0.01
nn = SimpleNN(vocab_size, embedding_dim, hidden_size, output_size)

true, pred = [], []
epochs = 100
for epoch in range(epochs):
    train_loss = 0.0
    true, pred = [], []  # Reset true and predicted labels for each epoch
    for x, y in zip(train_texts, train_labels):
        x = encode_sentence(x)  # Encode sentence as word indices
        train_loss += train(x, y, learning_rate)  # Train on the current sample
        _, prediction = predict(x)  # obtain prediction
        true.append(y)  # Append true label
        pred.append(prediction[0][0])  # Append predicted label (extract scalar)

    # Calculate training accuracy
    train_acc = accuracy_score(true, pred) * 100.0
    train_loss /= len(train_texts)  # Average training loss

    # Evaluate on dev set
    dev_true, dev_pred = [], []
    dev_loss = 0.0
    for x, y in zip(dev_texts, dev_labels):
        x = encode_sentence(x)

        output, prediction = predict(x)
        dev_loss += nn.compute_loss(y,output)
        dev_true.append(y)
        dev_pred.append(prediction[0][0])
    dev_loss /= len(dev_texts)
    dev_acc = accuracy_score(dev_true, dev_pred) * 100.0
    print(f'{epoch} epoch, train_loss = {train_loss:.4f}, train_acc: {train_acc:.2f}%, eval_loss: {dev_loss:.4f}, eval_acc: {dev_acc:.2f}%')


0 epoch, train_loss = 0.7121, train_acc: 65.00%, eval_loss: 0.6979, eval_acc: 50.00%
1 epoch, train_loss = 0.7110, train_acc: 63.00%, eval_loss: 0.6979, eval_acc: 50.00%
2 epoch, train_loss = 0.7109, train_acc: 63.00%, eval_loss: 0.6978, eval_acc: 50.00%
3 epoch, train_loss = 0.7109, train_acc: 62.00%, eval_loss: 0.6977, eval_acc: 50.00%
4 epoch, train_loss = 0.7108, train_acc: 62.00%, eval_loss: 0.6976, eval_acc: 50.00%
5 epoch, train_loss = 0.7107, train_acc: 62.00%, eval_loss: 0.6976, eval_acc: 50.00%
6 epoch, train_loss = 0.7106, train_acc: 62.00%, eval_loss: 0.6975, eval_acc: 50.00%
7 epoch, train_loss = 0.7106, train_acc: 62.00%, eval_loss: 0.6974, eval_acc: 50.00%
8 epoch, train_loss = 0.7105, train_acc: 62.00%, eval_loss: 0.6973, eval_acc: 50.00%
9 epoch, train_loss = 0.7104, train_acc: 62.00%, eval_loss: 0.6972, eval_acc: 50.00%
10 epoch, train_loss = 0.7103, train_acc: 62.00%, eval_loss: 0.6972, eval_acc: 50.00%
11 epoch, train_loss = 0.7103, train_acc: 62.00%, eval_loss: 0.6